In [ ]:
def minimal_graph(M):
    def unfill(M, check):
        n = M.nrows()
        M1 = copy(M)
        zero = Integer(0)
        for i in range(n):
            M1[i, i] = zero
            for j in range(i):
                if not check(M1[i, j]):
                    M1[i, j] = zero
                    M1[j, i] = zero
        return M1
    Mprime = unfill(M, is_prime)
    M2prime = Mprime**2
    M1 = copy(M)
    for i in range(M.nrows()):
        for j in range(i):
            if Mprime[i,j] != 0:
                continue
            if M1[i,j] != 0 and M2prime[i,j] != 0:
                M1[i,j] = M1[j,i] = 0
    return M1

from collections import defaultdict, Counter
def read_output_hash(filename):
    ZZx = PolynomialRing(ZZ, 'x')
    graphs = defaultdict(list)
    with open(filename, "r") as F:
        res = dict()
        for i, line in enumerate(F.readlines(), 1):
            s = line.strip().split(":")
            if len(s) != 9:
                continue
            _, _, h, _, _, ells, _, eqnso, M = s
            h = int(h)
            eqns = [[ZZx(elt) for elt in eqn] for eqn in eval(eqnso)]
            M = minimal_graph(matrix(eval(M)))
            G = Graph(M, weighted=True)
            for g in graphs[(G.num_verts(), G.num_edges())]:
                b, m = G.is_isomorphic(g, certificate=True)
                if b:
                    G.relabel(m)
                    eqns = [eqns[m[i]] for i,_ in enumerate(eqns)]
                    M = G.weighted_adjacency_matrix()
                    break
            else:
                graphs[(G.num_verts(), G.num_edges())].append(G)
            res[h] = (eqns, ells, M)
    return dict(res)
def read_output_graph(filename):
    ZZx = PolynomialRing(ZZ, 'x')
    graphs = defaultdict(list)
    with open(filename, "r") as F:
        res = defaultdict(list)
        for i, line in enumerate(F.readlines(), 1):
            s = line.strip().split(":")
            if len(s) != 9:
                continue
            c, _, h, _, _, ells, _, eqnso, M = s
            #eqns = [[ZZx(elt) for elt in eqn] for eqn in eval(eqnso)]
            c, h = map(int, (c, h))
            M = eval(M)
            if len(M) in [1,2]:
                res[tuple(sum(M, []))].append((c, h))
                continue
            M = matrix(M)
            M = minimal_graph(M)
            G = Graph(M, weighted=True)
            inv0 = (G.num_verts(), G.num_edges())
            for g in graphs[inv0]:
                if G.is_isomorphic(g, edge_labels=True):
                    M = g.weighted_adjacency_matrix()
                    break
            else:
                graphs[inv0].append(G)
            res[tuple(M.list())].append((c, h))
    return dict(res)

In [ ]:
%time data = read_output_graph("data/output_g2database_2e20.txt")

In [ ]:
%time out_hash = read_output_hash("data/output_g2database_2e20.txt")

In [ ]:
memory = []
with open("data/memory.log") as F:
    for i, line in enumerate(F.readlines()):
        memory.append(int(line)/1024.)
memory.sort()
print(memory[-10:])

In [ ]:
times = []
with open("data/joblog_g2database_2e20.txt.jun25") as F:
    for i, line in enumerate(F.readlines()):
        if i == 0:
            continue
        s = line.strip().split("\t")
        h = s[-1].split(':')
        times.append((float(s[3]), h[0].split(" ")[-1], h[1], int(h[2])))
times.sort()
morethan10min = [elt for elt in times if elt[0] > 616]
print(len(morethan10min))
long = defaultdict(list)
for elt in morethan10min:
    o = out_hash[elt[3]]
    isog = tuple(set(o[2].list()))
    long[(isog, tuple(eval(o[1])))].append((elt[0], o[2].nrows()))
for k, v in sorted(long.items()):
    print(k[0], k[1])
    print(len(v), v)
    print("")

In [ ]:
print(sum([elt[0] for elt in [(1019.898, 1), (1050.102, 1), (1051.597, 1), (1054.231, 1), (1061.672, 1), (1114.16, 1)]])/(6*60))
print(times[-2][0]/60)

In [ ]:
graphs_persize = defaultdict(list)
for k, v in data.items():
    n = sqrt(len(k))
    M = matrix(n, n, list(k))
    G = Graph(M, weighted=True)
    graphs_persize[sqrt(len(k))].append((G, len(v)))

In [ ]:
counts = {k : len(v) for k, v in data.items()}
# total number of curves
total = sum(v*sqrt(len(k)) for k, v in counts.items())
newcurves = total - 1743737
print("total =", total, "\nnew =", newcurves)

from collections import Counter
isogdeg = [(k, v/2) for k, v in sum([Counter(k*v) for k, v in counts.items()], Counter()).items() if k != 0]
isogdeg = [ (a^2, b) for a,b in isogdeg]
isogdeg.sort(key=lambda x: (prime_divisors(x[0]), x[0]))
for i in range(7):
    elt = isogdeg[i]
    print(rf'${latex(factor(elt[0]))}$ & \numprint{{{elt[1]}}}')
    if i + 7 < len(isogdeg):
        elt = isogdeg[i + 7]
        print(" && ")
        print(rf'${latex(factor(elt[0]))}$ & \numprint{{{elt[1]}}}\\')
        
# print(' & '.join([f'${latex(factor(elt[0]))}$' for elt in isogdeg]))
# print(' & '.join([r'\numprint{%s}' % elt[1] for elt in isogdeg]))

In [ ]:
sizes = sorted(graphs_persize)
for i in range(10):
    k = sizes[i]
    print(k, "&", r'\numprint{%s}' % sum([elt1[1] for elt1 in graphs_persize[k]]))
    if i + 10 < len(sizes):
        k = sizes[i+10]
        print(" && ")
        print(k, "&", r'\numprint{%s}\\' % sum([elt1[1] for elt1 in graphs_persize[k]]))

In [ ]:
%mkdir data/g2database_2e20

In [ ]:
for k in sorted(graphs_persize):
    for i, (G, lv) in enumerate(sorted(graphs_persize[k], key=lambda x: -x[1])):
            pl = G.plot(
            edge_labels=True,
            vertex_labels=False,
            iterations=10**5,
            spring=10,
            talk=True,
            #graph_border=True,
            figsize=[16,7]
            )
            pl.save(f"data/g2database_2e20/{k}-{i}-{lv}.png")
            pl.show()
            print(lv)
            print("#"*30)